#### 讀取需要的函式庫

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from func import *
import jieba
import re
import os
# re -> \d for number, \s for any space (\t\r\n...) 

#### 讀取資料夾中的資料

In [2]:
path = '../../107-1PythonSampleCode/week_5/task_5/*.txt'
title, data = readdir(path, 0.6)

Unknown encode file :
../../107-1PythonSampleCode/week_5/task_5/2015-4.txt
../../107-1PythonSampleCode/week_5/task_5/2015-all.txt

Number of unknown encoding files : 2
Number of read files : 4
Number of files : 6


#### 顯示部份資料內容

In [3]:
for i in range(len(data)):
    print('----Document : {0}----'.format(title[i]))
    print(data[i][:50], '...', data[i][-50:-1], sep='\n', end='\n\n')

----Document : 2012-4----
股票代碼:3548
兆利科技工業股份有限公司及其子公司
合 併 財 務 報 表
民國一○一年及一○○
...
587 客 戶 乙 318,057 358,352 $ 742,941 1,132,939
~40

----Document : 2014-4----

壹、致股束報告書
回顧-O三年，由於全球經濟景氣復甦緩慢，市場需求表現平淡，依Digitimes

...
及截至年報刊印日止之處理情形:無。

（十三）其他重要風險及因應措泥:無。七、其他重要事項:無。


----Document : 2016-4----

壹、致股東報告書
回顧一O五年,由於全球筆電品牌廠面臨關鍵零組件缺貨及漲價壓力,抑制筆記 型電腦整
...
款所定對股東權
益或證券價格有重大影響之事項:無。
- 221 -
_______________

----Document : 2016-all----
一???五年度 年 報
中華民國一???六年六月六日刊印
本年報內容及本公司相關資訊可至下列網址查詢
...
款所定對股東權
益或證券價格有重大影響之事項:無。
- 221 -
_______________



#### 新增詞彙至自定義詞庫，並匯入該詞庫
* savetofile(path, list_like_data)
* example for list_like_data:
  * [['來台']] : 完整 '來台' 字串被存入字典中
  * ['來台'] :　'來台' 視為獨立字元，存入 '來' '台' 於字典中 

In [4]:
savetofile('./dict.txt', [['來台']])  # 完整字串　'來台'　被存入字典中
jieba.load_userdict('./dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.137 seconds.
Prefix dict has been built succesfully.


#### 新增詞彙至自定義停止詞彙中

In [5]:
stopwords = ['一','二','三','四','五','六','用','淨','利','值','但','中','可','認','我','無','彙',
             '列','將','對','之','率','已','由','佔','本','日','月','年','另','故','於','較',
             '請','再','經','也','在','依','到','等','或','認列','為','按','比','仍','係','後','及',
             '與','以','時','該','的','並','更','註','例如','可能','由於','因此','附註','部分','事項',
             '各位','晶華','品牌','集團','幅度','資產','新','台幣','營業','受到','提供','使用','本年',
             '說明','現金','成本','營業外','收入','結轉','分攤','其他','影響','帳金額','本會','計師',
             '之會計','報表','會計師','財務','詳合','結果','預測','如下','之一','附註','附','註','其中',
             '併','合','因為','查核','報告','性事','增加','會計','相對','仟元','金額','權益','能力',
             '利益','股東','合理性','假設','無形','上','開關鍵','模型','支出','控制','計算','資金',
             '評估','報告','因應','仟元','金額','權益','假設','無形','上','模型','支出','控制','計算',
             '資金','評估','人次','重大','資訊','總額','民國','估計','','一副','公司','年度','包括',
             '正確性','正','確性','減損','共為','預估','預算','維持','公開','整體','表示','特性','市場',
             '科目','發生','年本','重要','判斷','主要','流量','意見','關鍵','同期','評價','正確','地區',
             '總資產','執行','程序','測試','稅後淨利','股份有限公司']
puncs_Full = '！？｡＂＃＄％＆＇（）()＊＋，－／：；＜＝＞＠［＼］$＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.。'
puncs_Half = "!?.#$%&'()*+,-/:;<=>@[]^_{}|~\\\""
savetofile('./stopword.txt', [stopwords])

In [6]:
stopword = list(map(str.strip, open('./stopword.txt', 'r+').readlines()))
stopword[0:10]

['一', '二', '三', '四', '五', '六', '用', '淨', '利', '值']

#### 利用 jieba 對文件進行斷字，並統計字詞頻率，顯示前十名資料

In [7]:
wordDictionary = []
for i in range(len(data)):
    seg_list = jieba.cut(data[i], cut_all=False)
    seg_list = list(seg_list)
    wordDict = {}
    for w in seg_list:
        if (w not in stopword) and (w not in puncs_Full) and (w not in puncs_Half):
            if w in wordDict:
                wordDict[w] = wordDict[w] + 1
            else:
                wordDict[w] = 1
    wordDictionary.append(wordDict)
    print('Number of word in document ({0}) : '.format(title[i]), len(wordDict))
    sort = sorted(wordDict.items(), key=lambda x: x[1], reverse = True)
    print(sort[0:10])
    data[i] = ' '.join(jieba.cut(data[i], cut_all=False))

Number of word in document (2012-4) :  2858
[(' ', 4198), ('\n', 928), ('1', 244), ('○', 189), ('3', 168), ('2', 143), ('兆利', 112), ('科技', 97), ('有限公司', 87), ('應', 71)]
Number of word in document (2014-4) :  4231
[('\t', 6817), ('\n', 3383), (' ', 3358), ('1', 467), ('2', 244), ('0', 224), ('損益', 201), ('金融', 181), ('3', 169), ('負債', 155)]
Number of word in document (2016-4) :  7316
[(' ', 11770), ('\n', 3770), ('1', 690), ('2', 400), ('3', 309), ('有限公司', 270), ('董事', 251), ('○', 245), ('科技', 215), ('105', 212)]
Number of word in document (2016-all) :  7779
[(' ', 14377), ('\n', 4217), ('1', 835), ('2', 524), ('3', 391), ('有限公司', 275), ('β', 274), ('董事', 259), ('○', 253), ('4', 249)]


#### 顯示各文件低於閥值 (limit) 的字詞

In [8]:
limit = 50
wordsperdocument = 50
for i in range(len(data)):
    count = 0
    print('----Document : {0}----'.format(title[i]))
    for ind, value in wordDictionary[0].items():
        if (value <= limit) & (count != wordsperdocument):
            count += 1
            print(ind, end = ', ')
    print()

----Document : 2012-4----
已達, 之間, 可靠, 登記, 首次, 末, 093, 租賃合, 認列淨, 7640, 據, 申請, 向下, 額以, 控制力, 期限, 並訂定, 143, 資源, 9850, 生效, 過渡, 管理, 相等, 943, 026, 一個, 之表達, 696, 011, 常熟, 7510, 961, 最近, USD8, 絲釘, 1.750, 限額, 全數, 073, 低於當, 可能性, 九十八, 修訂, 725, 領贈, 可回收, 並適, 負擔, 配股, 
----Document : 2014-4----
已達, 之間, 可靠, 登記, 首次, 末, 093, 租賃合, 認列淨, 7640, 據, 申請, 向下, 額以, 控制力, 期限, 並訂定, 143, 資源, 9850, 生效, 過渡, 管理, 相等, 943, 026, 一個, 之表達, 696, 011, 常熟, 7510, 961, 最近, USD8, 絲釘, 1.750, 限額, 全數, 073, 低於當, 可能性, 九十八, 修訂, 725, 領贈, 可回收, 並適, 負擔, 配股, 
----Document : 2016-4----
已達, 之間, 可靠, 登記, 首次, 末, 093, 租賃合, 認列淨, 7640, 據, 申請, 向下, 額以, 控制力, 期限, 並訂定, 143, 資源, 9850, 生效, 過渡, 管理, 相等, 943, 026, 一個, 之表達, 696, 011, 常熟, 7510, 961, 最近, USD8, 絲釘, 1.750, 限額, 全數, 073, 低於當, 可能性, 九十八, 修訂, 725, 領贈, 可回收, 並適, 負擔, 配股, 
----Document : 2016-all----
已達, 之間, 可靠, 登記, 首次, 末, 093, 租賃合, 認列淨, 7640, 據, 申請, 向下, 額以, 控制力, 期限, 並訂定, 143, 資源, 9850, 生效, 過渡, 管理, 相等, 943, 026, 一個, 之表達, 696, 011, 常熟, 7510, 961, 最近, USD8, 絲釘, 1.750, 限額, 全數, 073, 低於當, 可能性, 九十八, 修訂, 725, 領贈, 

#### 利用 wordDictionary 移除只出現一次的字詞，並顯示部份結果

In [9]:
for i in range(len(data)):
    for key, value in wordDictionary[0].items():
        if value <= limit:
            data[i] = data[i].replace(key, '')
    print('document :', title[i])
    print(data[i][:50], '...', data[i][-50:-1], sep='', end='\n\n')

document : 2012-4
  : 3 
 利 技  份    
 合   併             
  一 ○  及 一 ...          3 , 5    , 3   $    ,    1 , 2 ,  
 ~  

document : 2014-4

 壹 、 致 束   
 顧 - O  ， 由 於 全球 景氣  甦 緩慢 ， 場 求  平淡 ，...: 無 。 
 
 （  ） 其 重  及 應 措泥 : 無 。  、 其 重  : 無 。 
 

document : 2016-4

 壹 、 致 東  
 顧 一 O  , 由 於 全球 筆 牌 廠 臨鍵  缺及 漲 壓力 , 抑...  本法       對 東 
  或 券 格  重  之 : 無 。 
 -   1   - 


document : 2016-all
一 ? ? ? 五 年   年    
 中 一 ? ? ? 六年   刊印 
 本年  及 本  ...  本法       對 東 
  或 券 格  重  之 : 無 。 
 -   1   - 




#### 移除所有標點符號、非中文英文數字之資料

In [10]:
for i in range(len(data)):
    data[i] = re.sub('[^\u4e00-\u9fff ]', '', data[i])
#[^\u4e00-\u9fff A-Za-z0-9]

#### 印出部份斷字結果

In [11]:
for i in range(len(data)):
    print('document :', title[i])
    print(data[i][:20], ' ... ', data[i][-20:-1], sep='', end='\n\n')

document : 2012-4
     利 技  份     合    ...                    

document : 2014-4
 壹  致 束    顧     由 於 ...  措泥  無    其 重   無  

document : 2016-4
 壹  致 東   顧 一    由 於 ...   重  之  無          

document : 2016-all
一    五 年   年     中 一 ...   重  之  無          



#### 初始化 TFIDF 物件

In [12]:
tfidf_vectorizer = TfidfVectorizer(min_df=0.3, analyzer = 'word', stop_words = stopword)
#tfidf_vectorizer = CountVectorizer(stop_words = stopword)

#### 顯示初始化時輸入的停止詞彙 

In [13]:
print(tfidf_vectorizer.get_stop_words())

frozenset({'', '一', '等', '台幣', '意見', '無', '利', '或', '總額', '支出', '因此', '其中', '維持', '如下', '年', '發生', '請', '報表', '預算', '較', '併', '其他', '後', '性事', '整體', '主要', '按', '附註', '時', '結果', '用', '營業', '我', '依', '為', '因應', '年度', '率', '資產', '新', '會計師', '資訊', '認列', '佔', '各位', '表示', '列', '總資產', '日', '晶華', '資金', '正確', '包括', '的', '部分', '執行', '評價', '品牌', '認', '事項', '模型', '正', '市場', '受到', '由於', '分攤', '增加', '幅度', '特性', '稅後淨利', '集團', '程序', '本年', '中', '於', '與', '合', '預估', '由', '營業外', '例如', '重要', '但', '人次', '權益', '關鍵', '之一', '同期', '以', '彙', '比', '測試', '無形', '正確性', '科目', '帳金額', '二', '可', '該', '會計', '減損', '地區', '重大', '計算', '一副', '將', '附', '可能', '對', '使用', '能力', '假設', '現金', '註', '已', '共為', '之', '六', '利益', '股份有限公司', '詳合', '提供', '股東', '本', '報告', '收入', '預測', '再', '金額', '仟元', '仍', '公司', '之會計', '結轉', '月', '財務', '淨', '也', '及', '估計', '本會', '說明', '係', '查核', '四', '開關鍵', '民國', '三', '評估', '公開', '因為', '合理性', '確性', '五', '流量', '並', '另', '故', '在', '相對', '控制', '成本', '判斷', '影響', '上', '經', '計師', '年本', '值', '到', '更'})


#### 計算 TFIDF 詞頻矩陣

In [14]:
tfidf_matrix = tfidf_vectorizer.fit_transform(data)

#### 顯示詞頻矩陣大小

In [15]:
tfidf_matrix.shape

(4, 1282)

#### 顯示部份詞頻矩陣中的字詞

In [16]:
words = tfidf_vectorizer.get_feature_names()
print(words[0:50])

['一些', '一倍', '一千', '一口', '一季', '一新', '一方', '一旦', '一步', '一波', '一環', '一百', '一直', '一經', '一致', '一遭', '一部', '一龍', '三方', '三日', '三星', '三種', '三線', '三親', '上升', '上斷', '上本', '上游', '上漲', '世界', '世輝', '並依', '並列', '並加蓋', '並參酌', '並取', '並合', '並呈', '並宣導', '並將', '並對', '並引', '並應', '並擬', '並斷', '並於給', '並於部', '並檢', '並無', '並無給']


In [17]:
for i in range(len(data)):
    print('----Document : {0}----'.format(title[i]))
    for j in range(len(words)):
        if tfidf_matrix[i,j] > 0.2:
              print(words[j], tfidf_matrix[i,j])

----Document : 2012-4----
三日 0.8405501489252789
東莞旺 0.44062006894892286
百之 0.20013098783935213
----Document : 2014-4----
三日 0.3986739032972506
任何 0.3575993035232177
暴險 0.22756319315113854
----Document : 2016-4----
全球 0.20418440358904028
措施 0.22566011142837847
液晶 0.25254386759697084
獨立 0.21493095114635816
----Document : 2016-all----
措施 0.21037232733805436
液晶 0.23543479104498452
獨立 0.20037003493190172


In [18]:
dist = cosine_similarity(tfidf_matrix)
print(dist)

[[1.         0.38285041 0.22270346 0.21485176]
 [0.38285041 1.         0.65427831 0.64327289]
 [0.22270346 0.65427831 1.         0.96860655]
 [0.21485176 0.64327289 0.96860655 1.        ]]
